# 🔬 Model Exploration & Training

**Vendor Consistency Predictor** — XGBoost training pipeline with SHAP interpretability.

## Pipeline Overview
1. Generate realistic synthetic vendor-order data
2. Feature engineering & exploratory analysis
3. Train XGBoost regressor with cross-validation
4. Evaluate model performance (RMSE, MAE, R²)
5. SHAP feature importance analysis
6. Export trained model for the FastAPI service

---
## 1. Setup & Imports

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Optional: SHAP for model interpretability
try:
    import shap
    SHAP_AVAILABLE = True
    print('SHAP loaded successfully')
except ImportError:
    SHAP_AVAILABLE = False
    print('SHAP not installed — run: pip install shap')

# Reproducibility
np.random.seed(42)

# Plot style
sns.set_theme(style='whitegrid', palette='viridis')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

print('Environment ready ✅')

---
## 2. Synthetic Data Generation

We simulate **10,000 historical orders** across 200 vendors. The target variable `prep_time_minutes` is influenced by:
- **Order hour** — dinner rush (18–21) adds delay
- **Day of week** — weekends have higher variance
- **Item count** — more items = longer prep
- **Peak hour flag** — derived feature for rush periods
- **Historical delay average** — vendor-specific reliability signal

In [ ]:
N_ORDERS = 10_000
N_VENDORS = 200

# Generate vendor-level characteristics (some vendors are consistently slow)
vendor_base_delay = {vid: np.random.exponential(3.0) for vid in range(1, N_VENDORS + 1)}

# Generate order-level data
data = []
for _ in range(N_ORDERS):
    vendor_id = np.random.randint(1, N_VENDORS + 1)
    order_hour = np.random.choice(range(24), p=[
        0.01, 0.005, 0.005, 0.005, 0.005, 0.01, 0.02, 0.03,  # 0-7
        0.04, 0.04, 0.05, 0.07, 0.09, 0.07, 0.05, 0.04,      # 8-15
        0.05, 0.06, 0.08, 0.09, 0.07, 0.05, 0.03, 0.02       # 16-23
    ])
    day_of_week = np.random.randint(0, 7)
    item_count = np.random.randint(1, 12)
    is_peak_hour = 1 if order_hour in [12, 13, 18, 19, 20, 21] else 0
    historical_delay_avg = vendor_base_delay[vendor_id] + np.random.normal(0, 1)
    historical_delay_avg = max(0, historical_delay_avg)

    # Target: prep time with realistic dependencies
    prep_time = (
        8.0                                           # base prep time
        + item_count * 1.8                            # per-item cost
        + is_peak_hour * np.random.uniform(2, 6)      # rush hour penalty
        + historical_delay_avg * 0.7                  # vendor reliability
        + (1 if day_of_week >= 5 else 0) * 2.5        # weekend surge
        + np.random.normal(0, 3)                      # noise
    )
    prep_time = max(3, prep_time)  # minimum 3 minutes

    data.append({
        'vendor_id': vendor_id,
        'order_hour': order_hour,
        'day_of_week': day_of_week,
        'item_count': item_count,
        'is_peak_hour': is_peak_hour,
        'historical_delay_avg': round(historical_delay_avg, 2),
        'prep_time_minutes': round(prep_time, 2)
    })

df = pd.DataFrame(data)
print(f'Dataset: {df.shape[0]:,} orders across {df["vendor_id"].nunique()} vendors')
df.head(10)

---
## 3. Exploratory Data Analysis

In [ ]:
df.describe().round(2)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Distribution of prep time
axes[0, 0].hist(df['prep_time_minutes'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Distribution of Prep Time')
axes[0, 0].set_xlabel('Prep Time (minutes)')
axes[0, 0].axvline(df['prep_time_minutes'].mean(), color='red', linestyle='--', label=f'Mean: {df["prep_time_minutes"].mean():.1f} min')
axes[0, 0].legend()

# Prep time by hour
hourly = df.groupby('order_hour')['prep_time_minutes'].mean()
axes[0, 1].bar(hourly.index, hourly.values, color=sns.color_palette('viridis', len(hourly)))
axes[0, 1].set_title('Avg Prep Time by Hour')
axes[0, 1].set_xlabel('Order Hour')
axes[0, 1].set_ylabel('Avg Prep Time (min)')

# Prep time vs item count
axes[1, 0].scatter(df['item_count'], df['prep_time_minutes'], alpha=0.1, s=5)
item_means = df.groupby('item_count')['prep_time_minutes'].mean()
axes[1, 0].plot(item_means.index, item_means.values, color='red', linewidth=2, marker='o')
axes[1, 0].set_title('Prep Time vs Item Count')
axes[1, 0].set_xlabel('Item Count')
axes[1, 0].set_ylabel('Prep Time (min)')

# Correlation heatmap
corr = df[['order_hour', 'day_of_week', 'item_count', 'is_peak_hour', 'historical_delay_avg', 'prep_time_minutes']].corr()
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', ax=axes[1, 1], center=0)
axes[1, 1].set_title('Feature Correlation Matrix')

plt.tight_layout()
plt.savefig('eda_overview.png', bbox_inches='tight')
plt.show()
print('Saved: eda_overview.png')

---
## 4. Train/Test Split & XGBoost Training

In [ ]:
FEATURE_COLS = ['order_hour', 'day_of_week', 'item_count', 'is_peak_hour', 'historical_delay_avg']
TARGET = 'prep_time_minutes'

X = df[FEATURE_COLS]
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'Train: {X_train.shape[0]:,} samples')
print(f'Test:  {X_test.shape[0]:,} samples')

In [ ]:
# XGBoost with tuned hyperparameters
params = {
    'objective': 'reg:squarederror',
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 5,
    'reg_alpha': 0.1,
    'reg_lambda': 1.0,
    'random_state': 42,
    'n_jobs': -1
}

model = xgb.XGBRegressor(**params)
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=20
)

print('\nTraining complete ✅')

---
## 5. Model Evaluation

In [ ]:
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('=' * 45)
print('           MODEL EVALUATION RESULTS')
print('=' * 45)
print(f'  RMSE:  {rmse:.2f} minutes')
print(f'  MAE:   {mae:.2f} minutes')
print(f'  R²:    {r2:.4f}')
print('=' * 45)

# Cross-validation for robustness check
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')
print(f'\n5-Fold CV RMSE: {-cv_scores.mean():.2f} ± {cv_scores.std():.2f}')

In [ ]:
# Predicted vs Actual scatter
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Scatter plot
axes[0].scatter(y_test, y_pred, alpha=0.3, s=10, c='steelblue')
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2, label='Perfect prediction')
axes[0].set_xlabel('Actual Prep Time (min)')
axes[0].set_ylabel('Predicted Prep Time (min)')
axes[0].set_title(f'Predicted vs Actual  (R² = {r2:.3f})')
axes[0].legend()

# Residual distribution
residuals = y_test - y_pred
axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1].axvline(0, color='black', linestyle='--')
axes[1].set_xlabel('Residual (Actual - Predicted)')
axes[1].set_ylabel('Count')
axes[1].set_title(f'Residual Distribution  (MAE = {mae:.2f} min)')

plt.tight_layout()
plt.savefig('model_evaluation.png', bbox_inches='tight')
plt.show()
print('Saved: model_evaluation.png')

---
## 6. SHAP Feature Importance

SHAP (SHapley Additive exPlanations) provides model-agnostic, per-prediction feature attributions.
This is critical for stakeholder buy-in — they need to see **why** the model predicts a delay.

In [ ]:
if SHAP_AVAILABLE:
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_test)

    # Summary plot — global feature importance with direction
    print('SHAP Summary Plot — which features drive predictions?')
    shap.summary_plot(shap_values, X_test, show=False)
    plt.tight_layout()
    plt.savefig('shap_summary.png', bbox_inches='tight')
    plt.show()
    print('Saved: shap_summary.png')
else:
    # Fallback: XGBoost built-in feature importance
    print('Using XGBoost built-in feature importance (install shap for SHAP plots)')
    importance = model.feature_importances_
    feat_imp = pd.Series(importance, index=FEATURE_COLS).sort_values(ascending=True)
    
    fig, ax = plt.subplots(figsize=(8, 5))
    feat_imp.plot(kind='barh', ax=ax, color=sns.color_palette('viridis', len(feat_imp)))
    ax.set_xlabel('Feature Importance (Gain)')
    ax.set_title('XGBoost Feature Importance')
    plt.tight_layout()
    plt.savefig('feature_importance.png', bbox_inches='tight')
    plt.show()
    print('Saved: feature_importance.png')

In [ ]:
if SHAP_AVAILABLE:
    # Single prediction explanation — useful for debugging individual predictions
    sample_idx = 0
    print(f'Explaining prediction for test sample #{sample_idx}:')
    print(f'  Actual: {y_test.iloc[sample_idx]:.1f} min')
    print(f'  Predicted: {y_pred[sample_idx]:.1f} min')
    print()
    
    shap.force_plot(explainer.expected_value, shap_values[sample_idx], X_test.iloc[sample_idx], matplotlib=True)
    plt.tight_layout()
    plt.savefig('shap_force_plot.png', bbox_inches='tight')
    plt.show()
    print('Saved: shap_force_plot.png')

---
## 7. Export Model for FastAPI Service

Save the trained model in XGBoost's native JSON format. The FastAPI service (`app/model.py`) loads this file at startup.

In [ ]:
import os

MODEL_PATH = os.path.join('..', 'model.json')

# Save using XGBoost's Booster (matches the loading logic in app/model.py)
model.get_booster().save_model(MODEL_PATH)

# Verify the saved model loads correctly
booster = xgb.Booster()
booster.load_model(MODEL_PATH)
test_dmatrix = xgb.DMatrix(X_test.iloc[:1])
verify_pred = booster.predict(test_dmatrix)

print(f'Model saved to: {os.path.abspath(MODEL_PATH)}')
print(f'Model size: {os.path.getsize(MODEL_PATH) / 1024:.1f} KB')
print(f'Verification prediction: {verify_pred[0]:.2f} min')
print(f'Original prediction:     {y_pred[0]:.2f} min')
print(f'Match: {np.isclose(verify_pred[0], y_pred[0])} ✅')

---
## Summary

| Metric | Value |
|--------|-------|
| Training samples | 8,000 |
| Test samples | 2,000 |
| Features | 5 |
| Model | XGBoost Regressor |
| RMSE | See output above |
| R² | See output above |

**Key findings:**
- `item_count` and `historical_delay_avg` are the strongest predictors
- Peak hour effects are captured by `is_peak_hour`
- Weekend surge adds ~2.5 min on average

**Next steps:**
- Integrate with a real feature store (BigQuery → `data/features.sql`)
- Add confidence intervals via quantile regression
- Experiment with LightGBM / CatBoost for comparison
- Deploy model monitoring with prediction drift detection